In [3]:
import numpy_financial as npf
import pandas as pd
import numpy as np


In [8]:
total_installments = 14
current_installment_number = 7
full_chit_value = 700000
chit_frequency_per_year = 2
previous_installments = [50000,42000, 40000, 40000, 43000,38500]
bid_amount = 100000
winner_installment_amount=full_chit_value / total_installments

def get_prize_amount(full_chit_value, bid_amount, winner_installment_amount):
    return full_chit_value - bid_amount - winner_installment_amount


def create_cashflows(
    previous_installments,
    prize_amount,
    winner_installment_amount,
    current_installments_number,
    total_installments,
):
    future_winner_installments = [-winner_installment_amount] * (
        total_installments - (current_installments_number)
    )
    cashflows = (
        [-i for i in previous_installments]
        + [(prize_amount)]
        + future_winner_installments
    )
    print(cashflows)
    return cashflows


def calculate_annual_irr(cashflows, chit_frequency_per_year):
    irr_per_period = npf.mirr(cashflows)
    annual_irr = ((1 + irr_per_period) ** chit_frequency_per_year) - 1
    return annual_irr


prize_amount = get_prize_amount(full_chit_value, bid_amount, winner_installment_amount)
cashflows = create_cashflows(
    previous_installments,
    prize_amount,
    winner_installment_amount,
    current_installment_number,
    total_installments,
)
annual_irr = calculate_annual_irr(cashflows, chit_frequency_per_year)
print(f"Annual IRR for Bid Winner: {annual_irr:.2%}")


[-50000, -42000, -40000, -40000, -43000, -38500, 550000.0, -50000.0, -50000.0, -50000.0, -50000.0, -50000.0, -50000.0, -50000.0]


TypeError: mirr() missing 2 required positional arguments: 'finance_rate' and 'reinvest_rate'

In [ ]:
def scenario_analysis(current_installment_number, bid_amounts, winner_installment_amount=50000):
    """
    Perform scenario analysis for different bid amounts
    
    Parameters:
    - current_installment_number: The current installment number
    - bid_amounts: List of bid amounts to analyze
    - winner_installment_amount: Monthly installment for winner (default: 50000)
    
    Returns:
    - DataFrame with scenario analysis results
    """
    results = []
    
    for bid in bid_amounts:
        prize = get_prize_amount(full_chit_value, bid, winner_installment_amount)
        cf = create_cashflows(
            previous_installments,
            prize,
            winner_installment_amount,
            current_installment_number,
            total_installments
        )
        annual_return = calculate_annual_irr(cf, chit_frequency_per_year)
        
        results.append({
            'Bid Amount': bid,
            'Prize Amount': prize,
            'Annual IRR': annual_return,
            'Annual IRR (%)': f"{annual_return:.2%}"
        })
    
    return pd.DataFrame(results)

# Example usage
bid_scenarios = [80000, 90000, 100000, 110000, 120000]
scenario_df = scenario_analysis(current_installment_number, bid_scenarios)
print(scenario_df)

[-42000, -40000, -40000, -43000, 570000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]
[-42000, -40000, -40000, -43000, 560000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]
[-42000, -40000, -40000, -43000, 550000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]
[-42000, -40000, -40000, -43000, 540000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]
[-42000, -40000, -40000, -43000, 530000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]
   Bid Amount  Prize Amount  Annual IRR Annual IRR (%)
0       80000        570000    0.056488          5.65%
1       90000        560000    0.071672          7.17%
2      100000        550000    0.088172          8.82%
3      110000        540000    0.106253         10.63%
4      120000        530000    0.126272         12.63%


In [ ]:
import numpy_financial as npf
from scipy.optimize import fsolve

cashflows = [
    -50000,
    -42000,
    -40000,
    -40000,
    -43000,
    -38500,
    550000.0,
    -50000.0,
    -50000.0,
    -50000.0,
    -50000.0,
    -50000.0,
    -50000.0,
    -50000.0,
]

# Standard function often returns nan
print(f"Standard IRR: {npf.irr(cashflows)}")


# Robust Solution: Use fsolve with a guess
def npv_func(rate):
    # Calculate NPV manually: Sum( Cashflow / (1+r)^t )
    return sum([cf / ((1 + rate) ** t) for t, cf in enumerate(cashflows)])


# We guess -0.1 (negative 10%) because we know it's a loss
irr_solution = fsolve(npv_func, -0.1)[0]

print(f"Calculated IRR: {irr_solution * 100:.2f}%")


Standard IRR: nan
Calculated IRR: 7.42%


C:\Users\jagad\AppData\Local\Temp\ipykernel_28936\1615413717.py:32: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  irr_solution = fsolve(npv_func, -0.1)[0]


: 